# A Simple Example of usage

In this example I will go through creating the following entities in order:

- warehouse
- shipping zone
- product attributes with their values.
- categories
- product type
- products with variants and their stocks.

I will use Tea product as a fictive example.

In [7]:
from saleor_gql_loader import ETLDataLoader

In [8]:
# I generated a token for my app as explained in the README.md
# https://github.com/grll/saleor-gql-loader/blob/master/README.md
etl_data_loader = ETLDataLoader("LcLNVgUt8mu8yKJ0Wrh3nADnTT21uv")

In [10]:
# create a default warehouse
warehouse_id = etl_data_loader.create_warehouse()
warehouse_id

'V2FyZWhvdXNlOmI1MzVhMjUxLTE0NWMtNGRkYy05YmMzLWE3ODYwZmI2ZDVmNg=='

In [11]:
# create a default shipping zone associated
shipping_zone_id = etl_data_loader.create_shipping_zone(addWarehouses=[warehouse_id])
shipping_zone_id

'U2hpcHBpbmdab25lOjI='

In [13]:
# define my products usually extracted from csv or scraped...
products = [
    {
        "name": "tea a",
        "description": "description for tea a",
        "category": "green tea",
        "price": 5.5,
        "strength": "medium"
    },
    {
        "name": "tea b",
        "description": "description for tea b",
        "category": "black tea",
        "price": 10.5,
        "strength": "strong"
    },
    {
        "name": "tea c",
        "description": "description for tea c",
        "category": "green tea",
        "price": 9.5,
        "strength": "light"
    }
]

In [14]:
# add basic sku to products
for i, product in enumerate(products):
    product["sku"] = "{:05}-00".format(i)

In [16]:
# create the strength attribute
strength_attribute_id = etl_data_loader.create_attribute(name="strength")
unique_strength = set([product['strength'] for product in products])
for strength in unique_strength:
    etl_data_loader.create_attribute_value(strength_attribute_id, name=strength)

In [17]:
# create another quantity attribute used as variant:
qty_attribute_id =  etl_data_loader.create_attribute(name="qty")
unique_qty = {"100g", "200g", "300g"}
for qty in unique_qty:
    etl_data_loader.create_attribute_value(qty_attribute_id, name=qty)

In [18]:
# create a product type: tea
product_type_id = etl_data_loader.create_product_type(name="tea",
                                                      hasVariants=True, 
                                                      productAttributes=[strength_attribute_id],
                                                      variantAttributes=[qty_attribute_id])

In [19]:
# create categories
unique_categories = set([product['category'] for product in products])

cat_to_id = {}
for category in unique_categories:
    cat_to_id[category] = etl_data_loader.create_category(name=category)

cat_to_id

{'black tea': 'Q2F0ZWdvcnk6NQ==', 'green tea': 'Q2F0ZWdvcnk6Ng=='}

In [20]:
# create products and store id
for i, product in enumerate(products):
    product_id = etl_data_loader.create_product(product_type_id,
                                                name=product["name"],
                                                description=product["description"],
                                                basePrice=product["price"],
                                                sku=product["sku"],
                                                category=cat_to_id[product["category"]],
                                                attributes=[{"id": strength_attribute_id, "values": [product["strength"]]}],
                                                isPublished=True)
    products[i]["id"] = product_id

In [23]:
# create some variant for each product:
for product in products:
    for i, qty in enumerate(unique_qty):
        variant_id = etl_data_loader.create_product_variant(product_id,
                                                            sku=product["sku"].replace("-00", "-1{}".format(i+1)),
                                                            attributes=[{"id": qty_attribute_id, "values": [qty]}],
                                                            costPrice=product["price"],
                                                            weight=0.75,
                                                            stocks=[{"warehouse": warehouse_id, "quantity": 15}])